In [37]:
# ! pip install openai
# ! pip install google
# ! pip install llama-index-readers-web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 86.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 73.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = 'api-key-here'
client = OpenAI()

ChatCompletionMessage(content="In the realm of coding, a tale is told,\nOf a concept profound, timeless and bold.\nRecursion, a loop of mystical design,\nUnraveling solutions, layer by line.\n\nA function within itself, it calls,\nEchoing through the digital walls.\nLike a mirror reflecting its own face,\nRepetition with elegance and grace.\n\nInto the depths of problems it dives,\nSplitting challenges into smaller hives.\nEach iteration a step closer to the end,\nUntil the answers in clarity blend.\n\nLike a never-ending cycle it goes,\nThrough the maze of algorithms and throes.\nA dance of logic, a symphony rare,\nRecursion weaves a code's secret lair.\n\nSo embrace this concept, don't shy away,\nLet recursion guide you, night and day.\nFor in the world of programming's art,\nIt's the recursive song that sets you apart.", role='assistant', function_call=None, tool_calls=None)


In [32]:
setup_string = """
Your job is to return Google search queries, specifically Google Dorks, to find more
about a subject given a dumped text file of all of the information we currently know
about the subject. Not all of this information should be used in every query.
You can output up to twenty queries, and they should be formatted in a bullet point list
format, where every bullet point contains only the Google Dork search query and nothing
else (without quotes).

The end goal is to create an OSINT (open source intelligence) report about
the given subject. For example, if the user wants to find more about a person,
you would help them find out about their location, age, education, associated
institutions, relevant connections, and so forth.
""" # add osint flowchart

In [33]:
prompt = """
I want to find out more about the person Jai Sharma using Google Dorks. Here are
some things I know about him:
- Went to Monta Vista High School
- Is friends with Milind Maiti
- Might be associated with Berkeley
- Research oriented and possibly interested in Machine Learning, wrote a paper with Christopher Sun
"""

In [36]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": setup_string},
    {"role": "user", "content": prompt}
  ]
)

result = completion.choices[0].message.content
search_list = result.split("\n")
print(result)
print(search_list)

- "Jai Sharma" "Monta Vista High School" site:linkedin.com
- "Jai Sharma" "Milind Maiti" friends site:facebook.com
- "Jai Sharma" Berkeley site:instagram.com
- "Jai Sharma" "Christopher Sun" "Machine Learning" research paper filetype:pdf
- "Jai Sharma" "Machine Learning" site:scholar.google.com
- "Jai Sharma" "Berkeley" "Machine Learning" intitle:resume
- "Jai Sharma" "Research interests" site:researchgate.net
- "Jai Sharma" "Christopher Sun" site:arxiv.org
- "Jai Sharma" "Machine Learning" intitle:publication
- "Jai Sharma" "Google Scholar" "Christopher Sun" "Berkeley"
- "Jai Sharma" "Monta Vista High School" alumni site:linkedin.com
- "Jai Sharma" "Berkeley" "Machine Learning" intitle:thesis
- "Jai Sharma" "Christopher Sun" "Machine Learning" blog
- "Jai Sharma" "Research assistant" "Christopher Sun" site:researchgate.net
- "Jai Sharma" "Machine Learning" intitle:conference presentation
- "Jai Sharma" "Monta Vista High School" graduation year
- "Jai Sharma" "Machine Learning" "Berkel

In [28]:
from googlesearch import search

query = "Geeksforgeeks"

for j in search(query, tld="co.in", num=10, stop=10, pause=2):
    print(j)

https://www.geeksforgeeks.org/
https://in.linkedin.com/company/geeksforgeeks
https://twitter.com/geeksforgeeks?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor
https://twitter.com/geeksforgeeks/status/1807112825149833308?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://twitter.com/geeksforgeeks/status/1807104112074346947?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://twitter.com/geeksforgeeks/status/1807069362639249494?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://twitter.com/geeksforgeeks/status/1807060393480606178?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ
https://www.quora.com/What-is-your-review-of-GeeksforGeeks
https://apps.apple.com/in/app/geeksforgeeks-learn-coding/id1641848816


In [78]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import HTMLNodeParser
urls = ["https://docs.llamaindex.ai"]
documents = SimpleWebPageReader().load_data(urls)

for i in range(len(documents)):
  documents[i].text = " ".join(documents[i].text.split(" "))
  # documents[i].text = "\n".join(documents[i].text.split("\n"))
  print(documents[i].text)

# parser = HTMLNodeParser(
#     # tags=["p", "h1"],
#     chunk_lines=40,
#     chunk_lines_overlap=15,
#     max_chars=300,
#     )

# nodes = parser.get_nodes_from_documents(documents)
# for node in nodes:
#   print(node.text)
#   print(node.metadata)


<!doctype html>
<html lang="en" class="no-js">
  <head>
    
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width,initial-scale=1">
      
      
      
        <link rel="canonical" href="https://docs.llamaindex.ai/">
      
      
      
        <link rel="next" href="getting_started/concepts/">
      
      
      <link rel="icon" href="_static/assets/LlamaLogoBrowserTab.png">
      <meta name="generator" content="mkdocs-1.5.3, mkdocs-material-9.5.13">
    
    
      
        <title>LlamaIndex - LlamaIndex</title>
      
    
    
      <link rel="stylesheet" href="assets/stylesheets/main.7e359304.min.css">
      
        
        <link rel="stylesheet" href="assets/stylesheets/palette.06af60db.min.css">
      
      


    
    
      
    
    
      
        
        
        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
        <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Roboto:300,300i,400,400i,70

In [66]:
documents[0]

Document(id_='https://docs.llamaindex.ai', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n<!doctype html>\n<html lang="en" class="no-js">\n  <head>\n    \n      <meta charset="utf-8">\n      <meta name="viewport" content="width=device-width,initial-scale=1">\n      \n      \n      \n        <link rel="canonical" href="https://docs.llamaindex.ai/">\n      \n      \n      \n        <link rel="next" href="getting_started/concepts/">\n      \n      \n      <link rel="icon" href="_static/assets/LlamaLogoBrowserTab.png">\n      <meta name="generator" content="mkdocs-1.5.3, mkdocs-material-9.5.13">\n    \n    \n      \n        <title>LlamaIndex - LlamaIndex</title>\n      \n    \n    \n      <link rel="stylesheet" href="assets/stylesheets/main.7e359304.min.css">\n      \n        \n        <link rel="stylesheet" href="assets/stylesheets/palette.06af60db.min.css">\n      \n      \n\n\n    \n    \n      \n    \n    \n      \

In [73]:
print(nodes[0].text)

Home
Learn
Use Cases
Examples
Component Guides
Advanced Topics
API Reference
Open-Source Community
LlamaCloud
Home
Home
          




    High-Level Concepts
  





    Installation and Setup
  





    How to read these docs
  






    Starter Examples
  





            Starter Examples
          




    Starter Tutorial (OpenAI)
  





    Starter Tutorial (Local Models)
  








    Discover LlamaIndex Video Series
  





    Frequently Asked Questions (FAQ)
  







    Starter Tools
  








            Starter Tools
          




    RAG CLI
High-Level Concepts
Installation and Setup
How to read these docs
Starter Examples
Starter Examples
          




    Starter Tutorial (OpenAI)
  





    Starter Tutorial (Local Models)
Starter Tutorial (OpenAI)
Starter Tutorial (Local Models)
Discover LlamaIndex Video Series
Frequently Asked Questions (FAQ)
Starter Tools
Starter Tools
          




    RAG CLI
RAG CLI
Learn
Learn
          




    Using LLMs
  






   

In [82]:
documents[i].text = "\n".join(documents[i].text.split("\n"))
documents[i].text

'\n<!doctype html>\n<html lang="en" class="no-js">\n  <head>\n    \n      <meta charset="utf-8">\n      <meta name="viewport" content="width=device-width,initial-scale=1">\n      \n      \n      \n        <link rel="canonical" href="https://docs.llamaindex.ai/">\n      \n      \n      \n        <link rel="next" href="getting_started/concepts/">\n      \n      \n      <link rel="icon" href="_static/assets/LlamaLogoBrowserTab.png">\n      <meta name="generator" content="mkdocs-1.5.3, mkdocs-material-9.5.13">\n    \n    \n      \n        <title>LlamaIndex - LlamaIndex</title>\n      \n    \n    \n      <link rel="stylesheet" href="assets/stylesheets/main.7e359304.min.css">\n      \n        \n        <link rel="stylesheet" href="assets/stylesheets/palette.06af60db.min.css">\n      \n      \n\n\n    \n    \n      \n    \n    \n      \n        \n        \n        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>\n        <link rel="stylesheet" href="https://fonts.googleapi

In [75]:
documents[0].text

'\n<!doctype html>\n<html lang="en" class="no-js">\n  <head>\n    \n      <meta charset="utf-8">\n      <meta name="viewport" content="width=device-width,initial-scale=1">\n      \n      \n      \n        <link rel="canonical" href="https://docs.llamaindex.ai/">\n      \n      \n      \n        <link rel="next" href="getting_started/concepts/">\n      \n      \n      <link rel="icon" href="_static/assets/LlamaLogoBrowserTab.png">\n      <meta name="generator" content="mkdocs-1.5.3, mkdocs-material-9.5.13">\n    \n    \n      \n        <title>LlamaIndex - LlamaIndex</title>\n      \n    \n    \n      <link rel="stylesheet" href="assets/stylesheets/main.7e359304.min.css">\n      \n        \n        <link rel="stylesheet" href="assets/stylesheets/palette.06af60db.min.css">\n      \n      \n\n\n    \n    \n      \n    \n    \n      \n        \n        \n        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>\n        <link rel="stylesheet" href="https://fonts.googleapi

In [76]:
documents[0].text = " "

In [77]:
documents[0].text

' '

In [80]:
print(" ")